# [Assignment 5](https://ovgu-ailab.github.io/idl2023/assignment5.html)

Collaborative Work from Adrian Bremer and Philipp Reinig

## Setup

In [1]:
import tensorflow as tf
import numpy as np

2023-11-10 15:01:29.239376: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 15:01:29.382318: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 15:01:29.382369: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 15:01:29.382886: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 15:01:29.447043: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 15:01:29.448376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

## Preparing IMDB

In [15]:
num_words = 20000
(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=num_words)

In [16]:
sequence_lengths = [len(sequence) for sequence in train_sequences]

max_len = max(sequence_lengths)
print(max_len)
mean_len = int(np.mean(sequence_lengths))
print(mean_len)

2494
238


In [17]:
train_sequences_padded = tf.keras.preprocessing.sequence.pad_sequences(
    train_sequences, 
    maxlen=mean_len # max_len
)

In [18]:
one_hot_sequences_padded = tf.one_hot(indices=train_sequences_padded, depth=num_words)

In [19]:
train_data = tf.data.Dataset.from_tensor_slices((one_hot_sequences_padded, train_labels))

<tf.Tensor: shape=(2, 238, 10), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]]], dtype=float32)>

## Thoughts on parameters

- _TODO_